クライアント側(完全版)\
これから使うものは以下


In [3]:
import socket
import numpy as np
import cv2
import time
import warnings
import threading

In [4]:
def keyinputhandler(dic, lock): #入力を受け付ける関数、この関数の中にslack-botの入力を入れれば良さそう
    while True:
        k = input() #今は標準入力を受け付けるだけ
        if k=='q': #終了のために実行
            lock.acquire()
            dic['endflag'] = True
            lock.release()
            break
        elif k=='a': #強制的に画像を呼び出したい時に実行
            lock.acquire()
            dic['additional'] = True
            lock.release()

def send_recv_data(dic, lock): #データの要求、受け取り、画像の保存
    MSGLEN = 15116544
    soc = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    soc_flag = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    
    try:
        soc.connect(("192.168.0.12", 50000))
        soc_flag.connect(("192.168.0.12", 51001))
    except ConnectionRefusedError:
        print("refused")
        return
    else:
        print("接続完了")
    
    st = 1#データの要求flag
    addflag = 0 #強制的なデータの要求flag
    pre = time.time()
    now = time.time()
    while True:
        if now - pre >60: #前回の測定から60秒以上たっていたらデータを要求
            st = 1
        lock.acquire()
        if dic['endflag'] == True:
            sent = 0
            while sent==0:
                sent=soc_flag.send(b'end')#プログラムを終了するように命令する
                time.sleep(0.1)
            break
        if dic['additional']==True: #即座にデータを送るように要求する
            addflag = 1
            dic['additional']==False
        lock.release()
        
        
        #30秒経過または強制的な要求でデータを要求、このif文の中で画像(mogumogu.jpg)を受け取り保存をしているので
        #この中に予測の処理とslack-botの出力機構を作るのが良さそう。スレッド間のメモリの共有に関しては以下を参照...
        if st==1 or addflag==1:
            print("start comm.")
            sent = 0
            while sent==0:
                sent = soc_flag.send(b'demand')
                time.sleep(0.1)
            chunks = []
            bytes_recd = 0
            while bytes_recd < MSGLEN:
                chunk = soc.recv(min(MSGLEN - bytes_recd, 2048))
                if chunk == b'':
                    raise RuntimeError("socket connection broken")
                chunks.append(chunk)
                bytes_recd = bytes_recd + len(chunk)
            data = b''.join(chunks)

            data = np.fromstring(data,dtype=np.uint8)
            data = np.reshape(data, (1944, 2592, 3))
            cv2.imwrite("mogumogu.jpg",data)
            print("safely comm done.")
            
            addflag = 0
            if st == 1: #定期的な要求の時
                pre = time.time() #前回の測定時間を記録
                st = 0
        
        time.sleep(0.1)
        now = time.time()
        
    soc.close()
    soc_flag.close()
    
def func(): #学科PCとかで動かす、入力を受け付けるスレッド(thread0)とラズパイにデータを要求して画像を受け取り保存するスレッド(thread1)の並列処理
    warnings.simplefilter('ignore')
    dic = {'endflag':False, 'additional':False}#共有データを辞書型で保存する。slack-botの入力と出力が別スレッドになっているので使いそう
    lock = threading.Lock()#データの競合を起こさないためにあるスレッドでアクセス、書き込みしている時は排他制御をする(OS参照、実際の書き方はアクセスする時にacquire()とrelease()で挟むだけ)
    thread0 = threading.Thread(target=keyinputhandler, args=(dic,lock))
    thread1 = threading.Thread(target=send_recv_data, args=(dic,lock))
    thread0.start()
    thread1.start()
    thread0.join()
    thread1.join()
    print("all threads are done.")

In [ ]:
func()

ああ
ああ


Exception in thread Thread-5:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-4-f9af1c790606>", line 20, in send_recv_data
    soc.connect(("192.168.0.12", 50000))
TimeoutError: [Errno 110] Connection timed out

